In [8]:
import numpy as np
from sklearn.preprocessing import StandardScaler

In [9]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64) # квалификация репетитора

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # подходит или нет репетитор

In [10]:
scaler = StandardScaler()

In [11]:
for i in range(1, X.shape[0]):
    X[i] = scaler.fit_transform(X[i].reshape(-1,1)).reshape(1,-1)

### 1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - np.clip).

$$Logloss=-y \ln(p) - (1-y)\ln(1-p)$$

In [12]:
def calc_logloss(y, y_pred):
    err = np.mean(- y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [13]:
def calc_logloss(y, y_pred):
    err = np.mean(- y * np.log(np.clip(y_pred, 1e-5, 1)) - (1.0 - y) * 
                  np.log(np.clip((1.0 - y_pred), 1e-5, 1)))
    return err

In [14]:
np.log(0.5)

-0.6931471805599453

In [15]:
np.log(1e-2)

-4.605170185988091

In [16]:
np.log(np.clip(0, 1e-2, 1))

-4.605170185988091

In [17]:
calc_logloss(1, 0.00001)

11.512925464970229

### 2. Подберите аргументы функции eval_LR_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [18]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [19]:
def eval_LR_model(X, y, iterations, alpha=1e-4):
    w_dist = np.inf
    np.random.seed(42)
    w = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations + 1):
        
        
        z = np.dot(w, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
    
        w -= alpha * (1/n * np.dot((y_pred - y), X.T))        
#         if i % (iterations / 10) == 0:
#             print(i, w, err)

    return w, err, i

In [20]:
iters = [200, 500, 1000, 2000]
alphas = [1e-4, 1e-4, 1e-3, 1e-2, 1e-1, 1]

In [21]:
results = []
for iteration in iters:
    for alpha in alphas:
        w, err, i = eval_LR_model(X, y, iteration, alpha)
        results.append({'iters': iteration, 'alpha': alpha, 'logloss': err,})
for i in results:
    print(i)

{'iters': 200, 'alpha': 0.0001, 'logloss': 0.7612071473029914}
{'iters': 200, 'alpha': 0.0001, 'logloss': 0.7612071473029914}
{'iters': 200, 'alpha': 0.001, 'logloss': 0.7381659587792261}
{'iters': 200, 'alpha': 0.01, 'logloss': 0.5779942810068626}
{'iters': 200, 'alpha': 0.1, 'logloss': 0.41604957231150996}
{'iters': 200, 'alpha': 1, 'logloss': 0.273589567007154}
{'iters': 500, 'alpha': 0.0001, 'logloss': 0.7572558998540831}
{'iters': 500, 'alpha': 0.0001, 'logloss': 0.7572558998540831}
{'iters': 500, 'alpha': 0.001, 'logloss': 0.7025169340653326}
{'iters': 500, 'alpha': 0.01, 'logloss': 0.49992247954456925}
{'iters': 500, 'alpha': 0.1, 'logloss': 0.3516499478119723}
{'iters': 500, 'alpha': 1, 'logloss': 0.2224106578949519}
{'iters': 1000, 'alpha': 0.0001, 'logloss': 0.7507528681143767}
{'iters': 1000, 'alpha': 0.0001, 'logloss': 0.7507528681143767}
{'iters': 1000, 'alpha': 0.001, 'logloss': 0.6512037362503481}
{'iters': 1000, 'alpha': 0.01, 'logloss': 0.46129017053513033}
{'iters': 1

#### Вывод: при 2000 итерациях и $\alpha$ = 1 _logloss_ минимальный и равен 0.14

### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются веса, которые уже посчитаны функцией eval_LR_model и X, на выходе - массив y_pred_proba).

In [22]:
w, err, i = eval_LR_model(X, y, 2000, 1)
w

array([ 3.72326319, -8.10878236, -7.79840001, 18.57323654])

In [23]:
def calc_pred_proba(X, w):
    return sigmoid(w.dot(X))

In [24]:
y_pred_proba = calc_pred_proba(X, w)
y_pred_proba

array([0.3429153 , 0.02049536, 0.99999991, 0.09461118, 0.88693524,
       0.07049476, 1.        , 0.03636945, 0.53837445, 0.99999493])

In [25]:
print(f'true: {y}\npred: {np.array(list(map(round, y_pred_proba)))}')

true: [0 0 1 0 1 0 1 0 1 1]
pred: [0 0 1 0 1 0 1 0 1 1]


#### Если порог выбирать 0.5, то модель обучилась идеально для тестовой выборки

### 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются веса, которые уже посчитаны функцией eval_LR_model и X, на выходе - массив y_pred).

In [26]:
def calc_pred(X, w, target=0.5):
    result = sigmoid(w.dot(X))
    return np.array(list(map(lambda x: int(x>=target), result)))

In [27]:
y_pred = calc_pred(X, w)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

### 5. Посчитайте accuracy, матрицу ошибок, precision и recall, а также F1-score.

In [28]:
def get_confusion_matrix(y, y_pred):
    result = np.zeros((2,2))
    to_check = list(zip(y, y_pred))
    for i in range(len(to_check)):
        if to_check[i][0]==0 and to_check[i][1]==0:
            result[1][1] += 1
        elif to_check[i][0]==1 and to_check[i][1]==1:
            result[0][0] += 1
        elif to_check[i][0]==1 and to_check[i][1]==0:
            result[1][0] += 1
        else:
            result[0][1] += 1
    return result

In [29]:
confusion_matrix = get_confusion_matrix(y, y_pred)

In [30]:
def get_metrics(c_matrix):
    precision = c_matrix[0][0] / (c_matrix[0][0] + c_matrix[0][1])
    recall = c_matrix[0][0] / (c_matrix[0][0] + c_matrix[1][0])
    accuracy = (c_matrix[0][0] + c_matrix[1][1]) / (c_matrix[0].sum() + c_matrix[1].sum())
    f_1 = 2 * precision * recall / (precision + recall)
    return precision, recall, accuracy, f_1

In [31]:
get_metrics(confusion_matrix)

(1.0, 1.0, 1.0, 1.0)

In [32]:
y_pred_2 = calc_pred(X, w, 0.8)
c_matrix_2 = get_confusion_matrix(y, y_pred_2)
precision, recall, accuracy, f_1 = get_metrics(c_matrix_2)
print(f'precision = {precision}\nrecall = {recall}\naccuracy = {accuracy}\nf1 = {f_1}')
c_matrix_2

precision = 1.0
recall = 0.8
accuracy = 0.9
f1 = 0.888888888888889


array([[4., 0.],
       [1., 5.]])

### 6. Могла ли модель переобучиться? Почему?

#### Могла, поскольку мы не ограничивали веса модели

### 7. *Создайте функции eval_LR_model_l1 и eval_LR_model_l2 с применением L1 и L2 регуляризации соответственно.

In [33]:
def eval_LR_model_L2(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    w_dist = np.inf
    np.random.seed(42)
    w = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations + 1):
        
        
        z = np.dot(w, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
    
        w -= alpha * ((1/n * np.dot((y_pred - y), X.T)) + 2 * lambda_ * w)
        if i % (iterations / 10) == 0:
            print(i, w, err)

    return w, err, i

In [34]:
w, _, _ = eval_LR_model_L2(X, y, 2000, 1)

200 [ 0.73299873 -3.38317013 -1.93752529  6.18207661] 0.27358979824758417
400 [ 1.29788374 -4.23396013 -3.05648461  8.46700999] 0.23565963170003695
600 [ 1.75664718 -4.92776322 -3.94118191 10.30966223] 0.21122031799739122
800 [ 2.14667862 -5.53273402 -4.69302945 11.89729018] 0.19315395824872253
1000 [ 2.48593864 -6.07033682 -5.35074773 13.2982114 ] 0.1791283548719198
1200 [ 2.78581465 -6.55382915 -5.9362455  14.55273736] 0.16790700138808443
1400 [ 3.05429239 -6.99282906 -6.46428471 15.68894017] 0.1587201763817602
1600 [ 3.29723257 -7.39467283 -6.94556604 16.72764427] 0.1510548230711767
1800 [ 3.51905886 -7.76509291 -7.38814509 17.68479924] 0.14455496451433938
2000 [ 3.72318232 -8.10864331 -7.79823856 18.57288303] 0.13896623448660053


In [35]:
def eval_LR_model_L1(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    w_dist = np.inf
    np.random.seed(42)
    w = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations + 1):
        
        
        z = np.dot(w, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
    
        w -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * np.sign(w))
        if i % (iterations / 10) == 0:
            print(i, w, err)

    return w, err, i

In [36]:
w, _, _ = eval_LR_model_L1(X, y, 2000, 1)

200 [ 0.73300098 -3.38317506 -1.93752931  6.18208682] 0.2735896035014857
400 [ 1.29789115 -4.23397293 -3.05649824  8.46704057] 0.23565919288524909
600 [ 1.75666114 -4.92778639 -3.941208   10.30971954] 0.2112196228496254
800 [ 2.14669998 -5.53276937 -4.69306992 11.89737864] 0.1931530215432588
1000 [ 2.48596803 -6.07038574 -5.35080405 13.29833445] 0.17912719731175528
1200 [ 2.78585259 -6.5538928  -5.93631896 14.55289792] 0.16790564233886898
1400 [ 3.05433937 -6.99290846 -6.46437647 15.68914087] 0.15871863213663223
1600 [ 3.29728901 -7.39476886 -6.94567718 16.72788751] 0.15105310686699686
1800 [ 3.51912515 -7.76520641 -7.38827665 17.68508727] 0.1445530869943666
2000 [ 3.72325885 -8.10877501 -7.79839156 18.57321798] 0.13896420420240133
